# Blackjack

## Description

This notebook contains the code for a one player vs dealer version of blackjack. The player starts with 100 dollars, whilst the dealer starts with 10,000. The aim is to try and win all the dealers money without going broke yourself.

## Imports

In [1]:
from random import shuffle
from IPython.display import clear_output

### Card Features

In [2]:
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace': [11, 1]}

# Classes

## Card Class

In [3]:
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        return f'{self.rank} of {self.suit} ({values[self.rank]})'

## Deck Class

In [4]:
class Deck:
    def __init__(self):
        self.cards = []
        for i in suits:
            for j in ranks:
                self.cards.append(Card(i, j))
            
    def shuffle_cards(self):
        shuffle(self.cards)
        
    def deal(self):
        return self.cards.pop()
    
    def add_card(self, card):
        self.cards.append(card)

## Player Class

In [5]:
class Player:
    def __init__(self, cash, name):
        self.cards = []
        self.cash = cash
        self.name = name
        
    def get_card(self, card):
        self.cards.append(card)
        
    def return_card(self):
        return self.cards.pop()

    def recieve_cash(self, amount):
        self.cash += amount
        
    def give_cash(self, amount):
        self.cash += - amount
        
    def hand(self, hidden = False):
        if hidden:
            print(self.cards[0], end='.\n')
        else:
            print(*self.cards, sep=", ", end=".\n")
                
    def score(self):
        score, aces = 0, 0
        for i in self.cards:
            if i.rank == 'Ace':
                aces += 1
                score += 1
            else:
                score += values[i.rank]
        if aces > 0 and score < 12:
            score_1 = score + 10
            return [score, score_1]
        else:
            return [score]

## Game Functions

In [6]:
def show_hand_func(player):

    print(f"\n{player.name}'s cards are:", end =' ')
    player.hand()
    print(f"{player.name}'s score is: {player.score()}.")
    print("")

In [7]:
def bet_func(player, dealer):
    
    max_bet = min(player.cash, dealer.cash)
    print(f"You have ${player.cash} in your account. The {dealer.name} has ${dealer.cash} in their account. The maximum you can bet is ${max_bet}.")
     
    while True:
        try:
            bet = int(input(f"How much would you like to bet (please input an integer):  $"))
        except:
            print('Sorry, that is not a valid amount/bet.')
        else:
            if bet <= max_bet:
                print("")
                return bet
            else:
                print('Sorry, that is not a valid amount/bet.')

In [8]:
def bust_check_func(player):
    if min(player.score()) > 21:
        return True
    else:
        return False

In [9]:
def another_card_func(player, deck, skip = ''):
    answer = skip
    
    while answer not in ['y', 'n']:
        answer = input("Would you like another card ('Y' or 'N'):  ").lower()
    if answer == 'y':
        player.get_card(deck.deal())
        return True
    else:
        return False

In [10]:
def broke_check_func(player, dealer):
    
    if player.cash == 0:
        print(f'You are broke! GAME OVER!')
        return True
    elif dealer.cash == 0:
        print(f'{dealer.name} is broke! CONGRATULATIONS, YOU WON!')
        return True
    else:
        return False

In [11]:
def return_cards_to_deck(player, dealer, deck):
    
    for i in range(len(player.cards)):
        deck.add_card(player.return_card())
    for j in range(len(dealer.cards)):
        deck.add_card(dealer.return_card())

In [12]:
def end_game_func():
    stop_game = ''
    
    while stop_game not in ['yes', 'n']:
        stop_game = input("Would you like to end the game ('Yes' or 'N'):  ").lower()
    if stop_game == 'n':
        return False
    else:
        return True

## Play The Game

In [13]:
player_name = 'Player'
player_money = 100
dealer_name = 'House'
dealer_money = 10000

In [14]:
###################### CREATE OBJECTS#################
deck = Deck()
player = Player(player_money, player_name)
dealer = Player(dealer_money, dealer_name)

broke = False
while not broke:
    return_cards_to_deck(player, dealer, deck)
    deck.shuffle_cards()
    clear_output()
    
    ##################### DEAL CARDS AND DISPLAY HANDS ##############
    for i in range(2):
        player.get_card(deck.deal())
        dealer.get_card(deck.deal())

    print(f"The {dealer.name}'s face-up card is the:", end=" ")
    dealer.hand(True)
    show_hand_func(player)

    ################### TAKE BET #######################
    bet = bet_func(player, dealer)

    ################### PLAYER BUST CHECK #################
    player_bust = False

    while not player_bust:
        ################# PLAYER DRAW ANOTHER CARD #################
        new_card = another_card_func(player, deck)  
        if new_card:
            show_hand_func(player)
            player_bust = bust_check_func(player)
            if player_bust:
                print(f'{player.name} is bust! Round over!')    
        else:
            break      

    ################### DEALER BUST CHECK DRAW ANOTHER CARD #################  
    dealer_bust = False

    while (not dealer_bust) and (not player_bust):
        if max(dealer.score()) <= max(player.score()):
            another_card_func(dealer, deck, skip= 'y')
            dealer_bust = bust_check_func(dealer)
        else:
            show_hand_func(dealer)            
            print(f'{dealer.name} Won round!')
            break

    ###################### RETURN RESULT #####################    
    if dealer_bust:
        show_hand_func(dealer)        
        print(f"{dealer.name} is bust! {player.name} won round!")
        player.recieve_cash(bet)
        dealer.give_cash(bet)
        print(f"{'-'*124}\n\n{' '*38}${bet}.00 Added to {player.name}'s account.\n\n{' '*43}Account currently sits at:${player.cash}.00\n\n{'-'*124}")
    else:
        dealer.recieve_cash(bet)
        player.give_cash(bet)  
        print(f"{'-'*124}\n\n{' '*38}${bet}.00 Removed from {player.name}'s account.\n\n{' '*43}Account currently sits at:${player.cash}.00\n\n{'-'*124}")

    ########################### CHECK WHETHER DEALER OR PLAYER IS BROKE #################################
    broke = broke_check_func(player, dealer)
    if broke == False:
        broke = end_game_func()

The House's face-up card is the: Six of Hearts (6).

Player's cards are: Seven of Diamonds (7), Queen of Diamonds (10).
Player's score is: [17].

You have $100 in your account. The House has $10000 in their account. The maximum you can bet is $100.
How much would you like to bet (please input an integer):  $100

Would you like another card ('Y' or 'N'):  n

House's cards are: Six of Hearts (6), Ace of Spades ([11, 1]), Four of Spades (4).
House's score is: [11, 21].

House Won round!
----------------------------------------------------------------------------------------------------------------------------

                                      $100.00 Removed from Player's account.

                                           Account currently sits at:$0.00

----------------------------------------------------------------------------------------------------------------------------
You are broke! GAME OVER!
